In [1]:
import numpy as np
import serial
import Ascii_funct
from serial.tools import list_ports
from datetime import time
from datetime import datetime
import time
import thingsboard_json_formatter
import m3_thingsboard_mqtt

import requests
import json
# from six import print_
global ts
# global params

In [56]:
def events(dct):
    
    pck = dct['S']
    evmap = {}
    evmap['avgP'] = 'avgP%s' % pck
    evmap['avgR'] = 'avgR%s' % pck
    evmap['minP'] = 'minP%s' % pck
    evmap['minR'] = 'minR%s' % pck
    evmap['maxP'] = 'maxP%s' % pck
    evmap['maxR'] = 'maxR%s' % pck
    evmap['stdP'] = 'stdP%s' % pck
    evmap['stdR'] = 'stdR%s' % pck
    

    for key in list(evmap):
        dct[evmap.get(key)] = dct.pop(key,None)
    
    dct['utc'] = ts
    return dct
        

def create_dicts(t,phase):
    global ts
    
    mapping = {}
    mapping['p'] = 'pwr%s' %phase
    mapping['q'] = 'rpwr%s' %phase
    mapping['v'] = 'vlt%s' %phase
    mapping['i'] = 'cur%s' %phase
    mapping['s'] = 'apwr%s' %phase
    mapping['f'] = 'freq%s' %phase
    mapping['pF'] = 'cos%s' %phase
    mapping['cF'] = 'scre%s' %phase
    mapping['an'] = 'angle%s' %phase
    mapping['cE'] = 'cnrg%s' %phase
    mapping['pE'] = 'pnrg%s' %phase
    mapping['cR'] = 'crnrg%s' %phase
    mapping['pR'] = 'prnrg%s' %phase
        
    
    
    # create dictionary from list after mapping the descriptors
    line = dict(zip((mapping.get(item,item) for item in (v.split('=')[0] for v in t)),((v.split('=')[1]) for v in t)))
    line[list(line)[-1]] = line[list(line)[-1]][:-2]
    
    # convert 1/12000 utc to 1/1/1970 utc timestamp
    if 'utc' in line.keys():
        line['utc'] = int((datetime(2000,1,1)-datetime(1970,1,1)).total_seconds())+int(line['utc'])
        
    if 'S' in line.keys():
        if line['S']=='0':
            ts = line['utc']
            event=1
        else:
            line = events(line)
        
    return line


def read_line(s,acc_token):
    
    phmap = {}
    phmap['1'] = 'A'
    phmap['2'] = 'B'
    phmap['3'] = 'C'
    
    
    asc = s.readline().decode('ascii').replace(',', '.')
    if asc:
#         print(asc)
        if asc[0] != 's': return 0
        else:
            t = asc.split('&')
            if t[2][0]=='L':
                t[2] = 'L=' + t[2][1]
            else:
                print('EVENT')
                t[2] = 'P=' + t[2][1]
                t[3] = 'S=' + t[3][1]
            phase = phmap.get(t[2][2])

            line = create_dicts(t,phase)

            line.pop('L', None)
            line.pop('sN', None)
            line.pop('ms', None)
            line.pop('P', None)
            line.pop('S', None)
            
            line = {'ts':int(line['utc'])*1000,'values':{x: line[x] for x in line if x not in 'utc'}}
#             print(line)
            send_data(line,acc_token)
            return line
        

def send_data(payload,acc_token):
    global address 

    dev_token = 'fpgUMFyhpCS6i2fsRax5'  

    my_json = json.dumps(payload)
    print(my_json)
#     print('url address:',address + "api/v1/"+dev_token+"/telemetry")
    r2 = requests.post(url = address + "api/v1/"+dev_token+"/telemetry",
                      data=my_json, headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token})
    print(r2.status_code)
    return

In [59]:

global address 
address = "http://52.77.235.183:8080/"
        
r = requests.post(address + "api/auth/login",
                  json={'username': 'tenant@thingsboard.org', 'password': 'tenant'}).json()
acc_token = 'Bearer' + ' ' + r['token']
zerotime = datetime.now()

# read ports and try to recognize the specific usb port
port = dict((x[0], x[2]) for x in serial.tools.list_ports.comports())  # todo find the port of usb
com = None
for x in port:
    a = port[x].split(' ')
    for i in a:
        if i == "VID:PID=0451:16C8": 
            com = x

            
# interv = 10000
nsamples = 40 # number of samples at each window, given that sampling rate is at 10msec
thres = 2 #threshold in watts
interv = 10000 # interval for all variables in msec
lns = 'L1' # lines

# initialize serial port and send message to close previous connections or whatever                
s = serial.Serial(com, baudrate=115200, xonxoff=True)
s.write('-mz ade disag stat off\r\n'.encode())
s.close()
time.sleep(1)
s = serial.Serial(com, baudrate=115200, timeout=1.2, xonxoff=True)
s.write(b'-mz ade disag mode ascii\r\n')
s.readline().decode('ascii')

s.write(b'-mz ade disag stat pA\r\n') #
s.readline().decode('ascii')

s.write(b'-mz ade rta cfg %d,%d\r\n' % (nsamples,thres)) # 
s.readline().decode('ascii')

s.write(b'-mz ade disag int %d\r\n' % interv) # set reading interval
s.readline().decode('ascii')

s.write(b'-mz ade disag lines L1\r\n') #
s.readline().decode('ascii')

# s.write('-mz relay stat 1\r\n'.encode())
# s.readline().decode('ascii')
print('Start monitoring...')

time.sleep(1)

# m3_thingsboard_mqtt.setup(None)  # --->setup
# m3_thingsboard_mqtt.start()  # --->start

# time.sleep(1)
while(True):
    if (datetime.now()-zerotime).total_seconds()>3600:
        r = requests.post(address + "api/auth/login",
                  json={'username': 'tenant@thingsboard.org', 'password': 'tenant'}).json()
        acc_token = 'Bearer' + ' ' + r['token']
        zerotime = datetime.now()
        
    read_line(s,acc_token)
    

Start monitoring...
{"ts": 1602662554000, "values": {"vltA": "239.751", "curA": "0.018", "pwrA": "-0.0", "rpwrA": "3.7", "apwrA": "4.4", "freqA": "49.931", "cosA": "1.00", "screA": "1.973", "angleA": "56.3", "cnrgA": "28599", "pnrgA": "327", "crnrgA": "1944", "prnrgA": "0"}}
200
{"ts": 1602662564000, "values": {"vltA": "240.168", "curA": "0.018", "pwrA": "0.0", "rpwrA": "2.1", "apwrA": "4.3", "freqA": "49.951", "cosA": "1.00", "screA": "2.507", "angleA": "29.7", "cnrgA": "28599", "pnrgA": "327", "crnrgA": "1944", "prnrgA": "0"}}
200
{"ts": 1602662574000, "values": {"vltA": "239.479", "curA": "0.018", "pwrA": "-0.1", "rpwrA": "3.5", "apwrA": "4.3", "freqA": "49.951", "cosA": "1.00", "screA": "2.693", "angleA": "53.5", "cnrgA": "28599", "pnrgA": "327", "crnrgA": "1944", "prnrgA": "0"}}
200
{"ts": 1602662584000, "values": {"vltA": "239.820", "curA": "0.018", "pwrA": "-0.2", "rpwrA": "3.4", "apwrA": "4.4", "freqA": "49.951", "cosA": "1.00", "screA": "2.204", "angleA": "128.7", "cnrgA": "28

KeyboardInterrupt: 